# 1 - Run Struphy main file in a notebook

In this tutorial we will learn about the Struphy main execution file `struphy/models/main/main.py`. This file is executed from the console upon calling
```
    $ struphy run MODEL
```
Please visit https://struphy.pages.mpcdf.de/struphy/sections/userguide.html for detailed information about this command. In this tutorial, we shall

1. Import `struphy/models/main/main.py` and look at its functionality.
2. Import the parameter file `params_mhd_vlasov.yml` and change some parameters.
3. Understand the normalization of Struphy models (which units are used).
3. Run the model [LinearMHDVlasovCC](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.hybrid.LinearMHDVlasovCC) in the notebook (without invoking the console).

## Main execution file and parameters

In [ ]:
from struphy.models.main import main

help(main)

The function `main.py` has three mandatory arguments:

- `model_name`
- `parameters`
- `path_out`

In this example, we shall simulate the current coupling hybrid model [LinearMHDVlasovCC](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.hybrid.LinearMHDVlasovCC):

In [ ]:
model_name = 'LinearMHDVlasovCC'

 The simulation results will be stored in the Struphy installation path (obtained via `struphy -p` from the console) under the folder `io/out/tutorial_01/`.

In [ ]:
import os
import struphy

path_out = os.path.join(struphy.__path__[0], 'io/out', 'tutorial_01')

In Struphy, parameters are passed to a model via a dictionary that is stored in `.yml` format (the "parameter file").

Template parameter files for each model are available in the struphy installation path under the folder `io/inp/`. Let us check these out:

In [ ]:
inp_path = os.path.join(struphy.__path__[0], 'io/inp')

os.listdir(inp_path)

The file `tutorials/params_mhd_vlasov.yml` is the one we shall use in this tutorial. 

Let us import it with the `yaml` package and print the obtained dictionary:

In [ ]:
params_path = os.path.join(inp_path, 'tutorials', 'params_mhd_vlasov.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
    
parameters

We can clearly identify the 9 top-level keys mentioned in the [Struphy userguide](https://struphy.pages.mpcdf.de/struphy/sections/userguide.html#setting-simulation-parameters).

## Struphy normalization (units)

Let us understand the units used in Struphy (model normalization). 

In the present example, the geometry is a `Cuboid` with specific left and right boundaries (and thus side length) in each of the three space directions:

In [ ]:
parameters['geometry']

The question arises in which units of length these numbers are expressed. From the console, the units could be checked by typing
```
    $ struphy units -i tutorials/params_mhd_vlasov.yml LinearMHDVlasovCC
```
Here, two informations are passed, namely the parameter file (`params_mhd_vlasov.yml`) and the model name (`LinearMHDVlasovCC`). 

The latter is obvious because each Struphy model has its own specific normalization, stated in the model's documentation (and docstring). 

The former, however, is not obvious (parameters influence the units?). **Indeed, Struphy provides the flexibility that the units of each model can be influenced by the user via the parameter file.**

Let us check the relevant section in the dictionary:

In [ ]:
parameters['units']

Here, the user can set

1. the unit of length $\hat x$ in meter
2. the unit of the magnetic field strength $\hat B$ in Tesla
3. the unit of the number density $\hat n$ in $10^{20}$ $m^{-3}$.

In the above example we have $\hat x \approx 0.023\,m$, $\hat B = 1\,T$ and $\hat n = 10^{20}$ $m^{-3}$. 

 All other units, such as for velocity $\hat v$ or time $\hat t$ etc., are derived from the three basic units above. How is this achieved? In Struphy, each model has two built-in class methods:
 
- `velocity_scale`
- `bulk_species`

These have been set by the model developer (hard-coded) and cannot be changed by the user. They determine the derived units in the following way:

The `bulk_species` sets the mass number ($A$) and charge number ($Z$) to be used in the calculation of units:


In [ ]:
from struphy.models.hybrid import LinearMHDVlasovCC

print(LinearMHDVlasovCC.bulk_species())

parameters['fluid'][LinearMHDVlasovCC.bulk_species()]['phys_params']


The `velocity_scale` (partly) determines the velocity unit $\hat v$. It has been set by the model developer to one of the following:

1. speed of light, $\hat v = c$
2. Alfvén speed of the bulk species, $\hat v = v_\textnormal{A, bulk} = \sqrt{\hat B^2 / (m_\textnormal{bulk} \hat n \mu_0)}$
3. Cyclotron speed of the bulk species, $\hat v = \hat x \Omega_\textnormal{c, bulk}/(2\pi) = \hat x\, q_\textnormal{bulk} \hat B /(m_\textnormal{bulk}2\pi)$

In [ ]:
print(LinearMHDVlasovCC.velocity_scale())


The three possible velocities scales are entirely defined in terms of:

- the three units $\hat x$, $\hat B$, $\hat n$, which are provided by the user (who can thus also influence $\hat v$)
- the `bulk_species` (through $m_\textnormal{bulk} = m_\textnormal{proton} A$ and $q_\textnormal{bulk} = q_\textnormal{e}Z$). 

The associated time scale is then automatically given by
$$
 \hat t = \hat x / \hat v \,.
$$

 To summarize: qualitatively, the `velocity_scale` and the `bulk_species` are fixed within each model by the developer (hard-coded). Quantitatively, the values (here for the Alfvén speed and the MHD charge and mass) are set by the user through the parameter file. 

Please check out https://struphy.pages.mpcdf.de/struphy/sections/models.html#normalization for further discussion on the units used in Struphy. In this tutorial, instead of the console, we can inspect the units of our run also directly in this notebook:

In [ ]:
units, eq_params = LinearMHDVlasovCC.model_units(parameters, verbose=True)
units

Aside from the units, there are also the equation parameters `eq_params` returned, namely

- `alpha_unit` being the ratio of the unit plasma frequency to the unit cyclotron frequency
- `epsilon_unit` being the ratio of the unit angular frequency to the unit cyclotron frequency

for each species.

In [ ]:
eq_params

The side lengths of the `Cuboid` in our example are:

In [ ]:
print('s1 = ', (parameters['geometry']['Cuboid']['r1'] - parameters['geometry']['Cuboid']['l1']) * units['x'], 'm')
print('s2 = ', (parameters['geometry']['Cuboid']['r2'] - parameters['geometry']['Cuboid']['l2']) * units['x'], 'm')
print('s3 = ', (parameters['geometry']['Cuboid']['r3'] - parameters['geometry']['Cuboid']['l3']) * units['x'], 'm')

## Run Struphy main

Let us get back to the parameter file and change some entries in the parameter dictionary before we run the model.

The end time of 50 is too long for our example and we wish to simulate more particles-per-cell than 200 to have a higher resolution. Let us change these two parameters.

In [ ]:
parameters['time']['Tend'] = 1.5
parameters['kinetic']['energetic_ions']['markers']['ppc'] = 400

We are now ready to call the Struphy main file. A tutorial of how to post-process the generated simulation data is available [here](https://struphy.pages.mpcdf.de/struphy/doc/_build/html/tutorials/tutorial_02_postproc_standard_plotting.html). 

The console equivalent of the following command is
```
    $ struphy run LinearMHDVlasovCC -i tutorials/params_mhd_vlasov.yml -o tutorial_01/
```

In [ ]:
main(model_name, parameters, path_out)